In [1]:

import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU') # 내 컴에 장착된 GPU를 list로 반환
try:
    tf.config.experimental.set_memory_growth(gpu[0], True) # GPU Memory Growth를 Enable
except RuntimeError as e:
    print(e) # Error 발생하면 Error 내용 출력

2022-04-08 00:33:17.711082: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-08 00:33:18.424099: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-08 00:33:18.424567: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-08 00:33:18.444649: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-08 00:33:18.445023: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: NVIDIA GeForce RTX 3070 computeCapability: 8.6
coreClock: 1.77GHz coreCount: 46 deviceMemorySize: 7.77GiB deviceMemoryBandwidth: 417.29GiB/s
2022-04-08 00:33:18.445038: I tensorflow/stream_executor/platform/def

In [2]:
from summarizer.domain.model.video import Video
import matplotlib

In [3]:
video = Video(key="123",url="tests/2.mp4", algorithm="yolov3")
images = video._read_video()
idx, image = next(images)

length : 4625
width : 640
height : 360
fps : 25.0


2022-04-08 00:33:18.742693: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-08 00:33:18.742827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-08 00:33:18.743097: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: NVIDIA GeForce RTX 3070 computeCapability: 8.6
coreClock: 1.77GHz coreCount: 46 deviceMemorySize: 7.77GiB deviceMemoryBandwidth: 417.29GiB/s
2022-04-08 00:33:18.743116: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-08 00:33:18.743130: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-04-08 00:33:18.743135: I tensorflow/stream_executor/platform/

In [4]:

features = image.extract()
from summarizer.domain.model.feature import VideoFeature
videofeature = VideoFeature(key="123",features=features)

2022-04-08 00:33:19.867244: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-08 00:33:19.884928: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3699650000 Hz
2022-04-08 00:33:20.495445: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-04-08 00:33:21.631735: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2022-04-08 00:33:21.631889: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: /usr/local/cuda-11.0/bin/ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas locatio

In [ ]:
video = Video(key="123",url="tests/2.mp4", algorithm="yolov3")
list_of_feature = video.extract_feature()

video.shorten(video_feature=list_of_feature, must_include_feature=["person"])

In [ ]:
import boto3


def create_movie_table(dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.create_table(
        TableName='Feature',
        KeySchema=[
            {
                'AttributeName': 'key',
                'KeyType': 'HASH'  # Partition key
            },
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'key',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table


if __name__ == '__main__':
    movie_table = create_movie_table()
    print("Table status:", movie_table.table_status)

In [5]:
import boto3
from summarizer.infrastructure.repository.feature_repository import FeatureRepository
dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")
movie_table = dynamodb.Table('Feature')
repo = FeatureRepository(movie_table)

In [6]:
videofeature = VideoFeature(key="abc",features=features)
repo.put(videofeature)

In [7]:
movie_table.scan()

{'Items': [{'features': [{'name': 'backpack',
     'current_frame': Decimal('0'),
     'percentage_probability': Decimal('82.7314555644989'),
     'box_points': [Decimal('296'),
      Decimal('147'),
      Decimal('449'),
      Decimal('288')]},
    {'name': 'person',
     'current_frame': Decimal('0'),
     'percentage_probability': Decimal('99.8492419719696'),
     'box_points': [Decimal('273'),
      Decimal('61'),
      Decimal('480'),
      Decimal('383')]},
    {'name': 'backpack',
     'current_frame': Decimal('0'),
     'percentage_probability': Decimal('71.69930338859558'),
     'box_points': [Decimal('278'),
      Decimal('106'),
      Decimal('474'),
      Decimal('376')]}],
   'key': 'abc'}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': '7f684d47-e209-47fe-84ec-bbc474214e73',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 07 Apr 2022 15:33:55 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2788424575',
   'x-amzn-re

In [8]:
x = repo.get('abc')

In [9]:
print(x)

key='abc' features=[FrameFeature(current_frame=0, name='backpack', percentage_probability=82.7314555644989, box_points=[296, 147, 449, 288]), FrameFeature(current_frame=0, name='person', percentage_probability=99.8492419719696, box_points=[273, 61, 480, 383]), FrameFeature(current_frame=0, name='backpack', percentage_probability=71.69930338859558, box_points=[278, 106, 474, 376])]


In [ ]:
x = movie_table.get_item(Key={"key":'abc'})

In [ ]:
print(VideoFeature(**x['Item']))